<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko). Edited by Anna Tarelina (@feuerengel). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

# <center>Assignment #2. Optional part
## <center> Implementation of the decision tree algorithm
    
#  <center>  <font color = 'red'> Warning! </font>This is a very useful but ungraded assignment

In [219]:
import numpy as np
import pandas as pd
import scipy.stats as scipy_stats
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator
from sklearn.datasets import make_classification, make_regression, load_digits, load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from enum import Enum

Let's fix `random_state` (a.k.a. random seed) beforehand.

In [220]:
RANDOM_STATE = 17

**Implement the class `DecisionTree`**
**Specification:**
- the class is inherited from `sklearn.BaseEstimator`;
- class constructor has the following parameters: 
    `max_depth` - maximum depth of the tree (`numpy.inf` by default); 
    `min_samples_split` - the minimum number of instances in a node for a splitting to be done (2 by default); 
    `criterion` - split criterion ('gini' or 'entropy' for classification, 'variance' or 'mad_median' for regression; 'gini' by default);
    
    A functional to be maximized to find an optimal partition at a given node has the form
    $$Q(X, j, t) = F(X) - \dfrac{|X_l|}{|X|} F(X_l) - \dfrac{|X_r|}{|X|} F(X_r),$$
    where $X$ are samples at a given node, $X_l$ and $X_r$ are partitions of samples $X$ into two parts 
    with the following condition $[x_j < t]$, and $F(X)$ is a partition criterion.
    
    For classification: let $p_i$ be the fraction of the instances of the $i$-th class in the dataset $X$.
    
    'gini': Gini impurity $F(X) = 1 -\sum_{i = 1}^K p_i^2$.
    
    'entropy': Entropy $F(X) = -\sum_{i = 1}^K p_i \log_2(p_i)$.
    
    For regression: $y_j = y(x_j)$ - is a target for an instance $x_j$, $y = (y_1, \dots, y_{|X|})$ - is a target vector.
    
    'variance': Variance (mean quadratic deviation from average) $F(X) = \dfrac{1}{|X|} \sum_{x_j \in X}(y_j - \dfrac{1}{|X|}\sum_{x_i \in X}y_i)^2$
    
    'mad_median': Mean deviation from the median $F(X) = \dfrac{1}{|X|} \sum_{x_j \in X}|y_j - \mathrm{med}(y)|$
    
- the class has several methods: `fit`, `predict` and `predict_proba`;
- the`fit` method takes the matrix of instances `X` and a target vector `y` (`numpy.ndarray` objects) and returns an instance of the class `DecisionTree` representing the decision tree trained on the dataset `(X, y)` according to parameters set in the constructor; 
- the `predict_proba` method takes the matrix of instances `X` and returns the matrix `P` of a size `X.shape[0] x K`, where `K` is the number of classes and $p_{ij}$ is the probability of an instance in $i$-th row of `X` to belong to class $j \in \{1, \dots, K\}$.
- the `predict` method takes the matrix of instances `X` and returns a prediction vector; in case of classification, prediction for an instance $x_i$ falling into leaf $L$ will be the class, mostly represented among instances in $L$. In case of regression, it'll be the mean value of targets for all instances in leaf $L$.

In [317]:
def __probability_distr_dict(array):
    arr = np.array(array, dtype=int)
    arr.flatten()
    arr.sort()
    countedElements = Counter(arr)
    uniqueElements = list(dict.fromkeys(arr))
    uniqueElements.sort()
    prob_dict = {}
    elements_len = len(arr)

    for key in uniqueElements:
        prob_dict[key] = countedElements[key] / elements_len
    return prob_dict

def entropy(y):
    prob_dict = __probability_distr_dict(array=y)

    result = 0
    for key in prob_dict:
        result += prob_dict[key] * np.log2(prob_dict[key])
    return -1 * result if result != 0 else 0.0


def gini(y):
    prob_dict = __probability_distr_dict(array=y)

    result = 0
    for key in prob_dict:
        result += prob_dict[key] ** 2
    return 1 - result


def variance(y):
    arr = np.array(y, dtype=int)
    arr.flatten()
    a = [(x - (np.sum(arr) / len(arr))) ** 2 for x in arr]
    result = np.sum(a) / len(a)
    return result


def mad_median(y):
    arr = np.array(y, dtype=int)
    arr.flatten()
    a = [(x - np.median(arr)) for x in arr]
    result = np.sum(a) / len(a)
    return result


def best_partition_functional_result(X=list(), y=list(), F=gini):
    left_split = []
    right_split = []
    maximization_parameter = 0
    intermediate_param = None
    split_index = 0
    indicies = range(0,len(X))
    
    for i in indicies:
        current_left_split = y[0:i]
        current_right_split = y[i:len(y)]
        current_maximization_parameter = partition_functional_result(y,
                                                                     current_left_split,
                                                                     current_right_split,
                                                                     F)
        if maximization_parameter < current_maximization_parameter:
            left_split = current_left_split
            right_split = current_right_split
            maximization_parameter = current_maximization_parameter
            intermediate_param = X[i] if i == 0 else (X[i - 1] + X[i]) / 2
            split_index = i
    return (left_split,
            right_split,
            X[0:len(left_split)],
            X[len(left_split):len(y)],
            F(left_split),
            F(right_split),
            F(y),
            intermediate_param,
            split_index)

# note! This function applies for check target parameter (which is supposed to be binary one)
def partition_functional_result(Y, Y_l, Y_r, F=gini):
    return F(Y) - (len(Y_l) / len(Y)) * F(Y_r) - (len(Y_r) / len(Y)) * F(Y_r)


The `Node` class implements a node in the decision tree.

In [318]:
class Node():

    def __init__(self, feature_idx=0, threshold=0, labels=[], left=None, right=None, criterion=Criterion.GINI):
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.labels = labels
        self.left = left
        self.right = right
        self.criterion_type = criterion
        self.criterion_value = 0
        self.__margin = 0

    def __str__(self):
        if self.left != None:
            self.left.__margin = self.__margin + 4
        if self.right != None:
            self.right.__margin = self.__margin + 4
        margin = ' ' * self.__margin
        feature = margin + "feature idx: " + str(self.feature_idx)
        str_threshhold = margin + "threshold: " + str(self.threshold)
        str_type = margin + self.criterion_type + " = " + str(self.criterion_value)
        label = margin + "labels: " + str(self.labels[0] if len(self.labels) != 0 else "No label")
        left_node = margin + "left: \n" + ('' if self.left != None else margin + ' ' * 4) + str(self.left)
        right_node = margin + "right: \n" + ('' if self.right != None else margin + ' ' * 4) + str(self.right)
        return feature + '\n' +\
               label + '\n' +\
               str_threshhold + '\n' +\
               str_type + '\n' +\
               left_node + '\n' +\
               right_node

Let's determine the function for calculating a prediction in a leaf. For regression, let's take the mean for all values in a leaf, for classification - the most popular class in leaf.

In [321]:
class Criterion(Enum):
    ENTHROPY = 'enthropy'
    GINI = 'gini'
    VARIANCE = 'variance'
    MAD_MEDIAN = 'mad_median'

    def criterion_func(self):
        if self == Criterion.GINI:
            return gini
        elif self == Criterion.ENTHROPY:
            return entropy
        elif self == Criterion.VARIANCE:
            return variance
        else:
            return mad_median

class DecisionTree(BaseEstimator):
    
    def __init__(self,
                 max_depth=np.inf,
                 min_samples_split=2, 
                 criterion=Criterion.GINI,
                 debug=False):
        
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.debug = debug
        self.primaryNode = None
    
    def fit(self, X, y):
        self.primaryNode = self.__build(X_train=X, 
                                        Y_train=y,
                                        criterion=self.criterion)
        return self

    def predict(self, X):
#         for record in X:
        pass
        
    def predict_proba(self, X):
        pass
    
    def __build(self,
                X_train=pd.DataFrame(),
                Y_train=pd.DataFrame(),
                current_criterion_value=None,
                iteration=0,
                criterion=Criterion.GINI):

        Func = criterion.criterion_func()
        # root node
        t = Node(feature_idx=-1,
                 threshold=0,
                 left=None,
                 right=None,
                 criterion=criterion)
        t.criterion_type = str(criterion.value)

        best_params = None

        if (current_criterion_value != None and \
            (current_criterion_value <= 0.01 or iteration >= self.max_depth)) or \
            (len(X_train) == 0 or len(Y_train) == 0) or \
            (len(X_train) + len(Y_train) < self.min_samples_split):
            
            return t
        else:
            iteration += 1
#             print("iteration: " + str(iteration))
#             print("Y_train columns: " + str(Y_train.columns[0]))
            for column in X_train.columns:
                current_feature = X_train[column]
                frame = {column: current_feature, Y_train.columns[0]: Y_train[Y_train.columns[0]]}
                combined_feature = pd.DataFrame(frame)
                sorted_feature = combined_feature.sort_values(column)
                previous_result = sorted_feature.iloc[0][Y_train.columns[0]]

                current_result = None
                idx_to_check = []

                # index_list = list(sorted_feature.index.values)
                # print("Indexes list:")
                # print(index_list)
                # for index in list(sorted_feature.index.values):
                #     # current_row = sorted_feature.loc[index]
                #     # current_result = sorted_feature.iloc[index, 1]
                #     current_result = sorted_feature.loc[index][str(Y_train.columns[0])]
                #     if current_result != previous_result:
                #         idx_to_check.append(index)
                #     previous_result = current_result
                #
                # #             print(idx_to_check)
                # if len(idx_to_check) == 0:
                #     break
                # calculate entropy gain or giny impyty reduction for current feature
                part_result = best_partition_functional_result(list(sorted_feature[column]),
                                                               list(sorted_feature[Y_train.columns[0]]),
                                                               F=Func)
                
                mean_gain_paramter = (part_result[4] + part_result[5]) / 2
                if best_params == None or (best_params[4] + best_params[5]) / 2 > mean_gain_paramter:
                    best_params = part_result

                    t.feature_idx = X_train.columns.get_loc(column)
                    t.labels = [column]
                    current_criterion_value = best_params[6]
                    t.criterion_value = current_criterion_value

            combined_data_set = pd.concat([X_train, Y_train], axis=1, sort=False)
            combined_data_set.sort_values(t.labels[0], inplace=True)

            t.threshold = best_params[7]
            X_left = combined_data_set.iloc[0:best_params[8], np.arange(0, len(X_train.columns))]
            X_right = combined_data_set.iloc[best_params[8]:len(combined_data_set),
                      np.arange(0, len(X_train.columns))]
            Y_left = pd.DataFrame(combined_data_set.iloc[0:best_params[8], len(X_train.columns)])
            Y_right = pd.DataFrame(combined_data_set.iloc[best_params[8]:len(combined_data_set),
                                   len(X_train.columns)])
#             print("X_left")
#             print(X_left)
#             print("X_right")
#             print(X_right)
#             print("Y_left")
#             print(Y_left)
#             print('Y_right')
#             print(Y_right)
            t.left = self.__build(X_left,
                                  Y_left,
                                  current_criterion_value=(best_params[4] + best_params[5]) / 2,
                                  iteration=iteration,
                                  criterion=criterion)
            t.right = self.__build(X_right,
                                   Y_right,
                                   current_criterion_value=(best_params[4] + best_params[5]) / 2,
                                   iteration=iteration,
                                   criterion=criterion)
        return t


In [350]:
df_common = pd.DataFrame({'Возраст':  [17, 64, 18, 20, 38, 49, 55, 25, 29, 31, 33],
                          'Зарплата': [25, 80, 22, 36, 37, 59, 74, 70, 33, 102, 88],
                          'Невозврат кредита': [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1]})

tree = DecisionTree(max_depth=4, min_samples_split=2, criterion=Criterion.GINI)
trained_tree = tree.fit(X=df_common.drop(columns=['Невозврат кредита'], axis=0),
                        y=df_common.drop(columns=['Возраст', 'Зарплата'], axis=0))
# print(trained_tree.primaryNode)

# for record in df_common['Возраст', 'Зарплата']:
#     print(record)

# df_common.values[0]
df_common[['Возраст', 'Зарплата']][df_common['Зарплата'] > 70].values[0]

array([64, 80])

## Testing the implemented algorithm

### Classification

Download the dataset `digits` using the method `load_digits`. Split the data into train and test with the `train_test_split` method, use parameter values `test_size=0.2`, and `random_state=17`. Try to train shallow decision trees and make sure that gini and entropy criteria return different results.

In [157]:
# You code here

Using 5-folds cross-validation (`GridSearchCV`) pick up the optimal values of the `max_depth` and `criterion` parameters. For the parameter `max_depth` use range(3, 11), for criterion use {'gini', 'entropy'}. Quality measure is `scoring`='accuracy'.

In [7]:
# You code here

Draw the plot of the mean quality measure `accuracy` for criteria `gini` and `entropy` depending on `max_depth`.

In [8]:
# You code here

**1. Choose all correct statements:**
1. Optimal value of the `max_depth` parameter is on the interval [4, 9] for both criteria.
2. Created plots have no intersection on the interval [3, 10]
3. Created plots intersect each other only once on the interval [3, 10].
4. The best quality for `max_depth` on the interval [3, 10] is reached using `gini` criterion .
5. Accuracy is strictly increasing at least for one of the criteria, when `max_depth` is also increasing on the interval [3, 10]

**2. What are the optimal values for max_depth and criterion parameters?**
1. max_depth = 7, criterion = 'gini';
2. max_depth = 7, criterion = 'entropy';
3. max_depth = 10, criterion = 'entropy';
4. max_depth = 10, criterion = 'gini';
5. max_depth = 9, criterion = 'entropy';
6. max_depth = 9, criterion = 'gini';

Train decision tree on `(X_train, y_train)` using the optimal values of `max_depth` and `criterion`. Compute class probabilities for `X_test`.

In [9]:
# You code here

Using the given matrix, compute the mean class probabilities for all instances in `X_test`.

In [10]:
# You code here

**3. What is the maximum probability in a resulted vector?**
1. 0.127
2. 0.118
3. 1.0
4. 0.09

## Regression

Download the dataset `boston` using the method `load_boston`. Split the data into train and test with the `train_test_split` method, use parameter values `test_size=0.2`, `random_state=17`. Try to train shallow regression decision trees and make sure that `variance` and `mad_median` criteria return different results.

In [11]:
# You code here

Using 5-folds cross-validation (`GridSearchCV`) pick up the optimal values of the `max_depth` and `criterion` parameters. For the parameter `max_depth` use `range(2, 9)`, for `criterion` use {'variance', 'mad_median'}. Quality measure is `scoring`='neg_mean_squared_error'.

In [12]:
# You code here

Draw the plot of the mean quality measure `neg_mean_squared_error` for criteria `variance` and `mad_median` depending on `max_depth`.

In [13]:
# You code here

**4. Choose all correct statements:**
1. Created plots have no intersection on the interval [2, 8].
2. Created plots intersect each other only once on the interval [2, 8].
3. Optimal value of the `max_depth` for each of the criteria is on the border of the interval [2, 8].
4. The best quality at `max_depth` on the interval [2, 8] is reached using `mad_median` criterion.

**5. What are the optimal values for `max_depth` and `criterion` parameters?**
1. max_depth = 9, criterion = 'variance';
2. max_depth = 5, criterion = 'mad_median';
3. max_depth = 4, criterion = 'variance';
4. max_depth = 2, criterion = 'mad_median';
5. max_depth = 4, criterion = 'mad_median';
6. max_depth = 5, criterion = 'variance'.